In [ ]:
import os, sys, pickle

from tqdm import tqdm

from matplotlib import pyplot as plt
plt.rcParams.update({
	"text.usetex": True,
	# "text.usetex": False,
	"text.latex.preamble": r"\usepackage{amsmath}",
	"font.size": 14,
})
print(plt.get_backend())

sys.path.append(os.path.join("..", "training"))

agg


In [ ]:
also_pdf = True
close_figs = False

T_R_ref = 126
T_J_ref = 120

gamma = 0.99
n_IC_per_replay = 200

n_closed_loops = 25
if n_closed_loops > 1:
	plt.switch_backend('agg')
	close_figs = True
else:
	plt.switch_backend('inline')
	close_figs = False

lr = 1e-2
parameterization = "low"
agent_update = 50

methods_list = ["untrained", f"gauss_newton_rad_{10 * lr:.1e}", os.path.join("sb3_td3", "lr_1.00e-03_bs_100000_batch_1024_tau_1.0e-01_pd_10_msv_1")]
labels = ["Untrained (MPC)", "Gauss-Newton (MPC)", "SB3 TD3 (NN)"]
linestyles = ["solid", "--", "-."]
colors = ["C0", "C1", "C2"]

closed_loop_basepath = os.path.join("..", "data", "closed_loops", f"gamma{gamma:.3f}, n_IC_per_replay{n_IC_per_replay}")
fig_basepath = os.path.join("..", "data", "closed_loops", f"gamma{gamma:.3f}, n_IC_per_replay{n_IC_per_replay}", "figs")

if not os.path.exists(fig_basepath):
	os.makedirs(fig_basepath)

In [ ]:
def plot_closed_loop_trajectories(clc: dict, labels: list[str], figpath: str):
	ncols = 2
	nrows = 3
	scaling_factor = 0.66
	alpha = 0.25

	figsize = (4 * ncols * scaling_factor, 3.0 * nrows * scaling_factor)
	
	fig, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize = figsize, constrained_layout = True, sharex=True)

	t_min = 0
	t_max = 0
	for (method, data), linestyle, color, label in zip(clc.items(), linestyles, colors, labels):
		
		time = data.time
		t_min = min(t_min, time[0])
		t_max = max(t_max, time[-1])

		C_A = data.y[:, 0]
		C_B = data.y[:, 1]
		T_R = data.y[:, 2]
		T_J = data.y[:, 3]

		F_in = data.u[:, 0]
		Q_dot = data.u[:, 1] * 1e-3

		ax[0, 0].plot(data.time, T_R, linestyle=linestyle, color=color, label = label)
		ax[1, 0].plot(data.time, T_J, linestyle=linestyle, color=color, label = label)
		ax[2, 0].step(data.time[:-1], F_in, linestyle=linestyle, color=color, label = label)
		ax[0, 1].plot(data.time, C_A, linestyle=linestyle, color=color, label = label)
		ax[1, 1].plot(data.time, C_B, linestyle=linestyle, color=color, label = label)
		ax[2, 1].step(data.time[:-1], Q_dot, linestyle=linestyle, color=color, label = label)


	# Plot the setpoints
	ax[0, 0].axhline(T_R_ref, color="black", linestyle="dotted", label=r"Setpoint")
	ax[1, 0].axhline(T_J_ref, color="black", linestyle="dotted", label=r"Setpoint")
	
	# Draw the constraints
	ax_limit_factor = 0.05
	C_A_min = 0.1
	C_A_max = 2.0
	delta_C_A = (C_A_max - C_A_min) * ax_limit_factor  * 2 
	C_B_min = 0.1
	C_B_max = 2.0
	delta_C_B = (C_B_max - C_B_min) * ax_limit_factor  * 2
	T_R_min = 80
	T_R_max = 140
	delta_T_R = (T_R_max - T_R_min) * ax_limit_factor  * 2
	T_J_min = 80
	T_J_max = 140
	delta_T_J = (T_J_max - T_J_min) * ax_limit_factor  * 2
	F_in_min = 5
	F_in_max = 40
	delta_F_in = (F_in_max - F_in_min) * ax_limit_factor  * 2
	Q_dot_min = -8.5
	Q_dot_max = 0
	delta_Q_dot = (Q_dot_max - Q_dot_min) * ax_limit_factor  * 2

	ax[0, 0].fill_between([t_min, t_max], T_R_min, T_R_min - delta_T_R, color = "tab:red", alpha = alpha, label = "Constraints")
	ax[0, 0].fill_between([t_min, t_max], T_R_max + delta_T_R, T_R_max, color = "tab:red", alpha = alpha)
	ax[1, 0].fill_between([t_min, t_max], T_J_min, T_J_min - delta_T_J, color = "tab:red", alpha = alpha, label = "Constraints")
	ax[1, 0].fill_between([t_min, t_max], T_J_max + delta_T_J, T_J_max, color = "tab:red", alpha = alpha)
	ax[0, 1].fill_between([t_min, t_max], C_A_min, C_A_min - delta_C_A, color = "tab:red", alpha = alpha, label = "Constraints")
	ax[0, 1].fill_between([t_min, t_max], C_A_max + delta_C_A, C_A_max, color = "tab:red", alpha = alpha)
	ax[1, 1].fill_between([t_min, t_max], C_B_min, C_B_min - delta_C_B, color = "tab:red", alpha = alpha, label = "Constraints")
	ax[1, 1].fill_between([t_min, t_max], C_B_max + delta_C_B, C_B_max, color = "tab:red", alpha = alpha)
	ax[2, 0].fill_between([t_min, t_max], F_in_min, F_in_min - delta_F_in, color = "tab:red", alpha = alpha, label = "Constraints")
	ax[2, 0].fill_between([t_min, t_max], F_in_max + delta_F_in, F_in_max, color = "tab:red", alpha = alpha)
	ax[2, 1].fill_between([t_min, t_max], Q_dot_min, Q_dot_min - delta_Q_dot, color = "tab:red", alpha = alpha, label = "Constraints")
	ax[2, 1].fill_between([t_min, t_max], Q_dot_max + delta_Q_dot, Q_dot_max, color = "tab:red", alpha = alpha)


	# Set the y-labels
	ax[0, 0].set_ylabel(r"$T_\mathrm{R} ~\left[^\circ\mathrm{C}\right]$")
	ax[1, 0].set_ylabel(r"$T_\mathrm{J} ~\left[^\circ\mathrm{C}\right]$")
	ax[2, 0].set_ylabel(r"$F_\mathrm{in} ~\left[\mathrm{h}^{-1}\right]$")
	ax[0, 1].set_ylabel(r"$C_\mathrm{A} ~\left[\mathrm{mol} \, \mathrm{L}^{-1}\right]$")
	ax[1, 1].set_ylabel(r"$C_\mathrm{B} ~\left[\mathrm{mol} \, \mathrm{L}^{-1}\right]$")
	ax[2, 1].set_ylabel(r"$\dot{Q} ~\left[\mathrm{kW}\right]$")

	# Adjust the y-lim
	ax[0, 0].set_ylim([T_R_min - delta_T_R / 2, T_R_max + delta_T_R / 2])
	ax[1, 0].set_ylim([T_J_min - delta_T_J / 2, T_J_max + delta_T_J / 2])
	ax[0, 1].set_ylim([C_A_min - delta_C_A / 2, C_A_max + delta_C_A / 2])
	ax[1, 1].set_ylim([C_B_min - delta_C_B / 2, C_B_max + delta_C_B / 2])
	ax[2, 0].set_ylim([F_in_min - delta_F_in / 2, F_in_max + delta_F_in / 2])
	ax[2, 1].set_ylim([Q_dot_min - delta_Q_dot / 2, Q_dot_max + delta_Q_dot / 2])


	handles, labels = ax[0, 0].get_legend_handles_labels()
	fig.legend(handles=handles, labels=labels, loc='outside lower center', ncol=2, fontsize=12)

	
	ax[-1, 0].set_xlabel(r"Time $t ~\left[\mathrm{h}\right]$")
	ax[-1, -1].set_xlabel(r"Time $t ~\left[\mathrm{h}\right]$")
	ax[-1, -1].set_xlim([t_min, t_max])

	plt.savefig(figpath, dpi = 300.0)
	if also_pdf:
		figpath_pdf = figpath.replace(".png", ".pdf")
		plt.savefig(figpath_pdf, dpi = 1200.0)

	if close_figs:
		plt.close("all")
	return

In [28]:
plt.close("all")
for n_ic in tqdm(iterable= range(n_closed_loops)):
	closed_loops = {}
	for method in methods_list:
		clc_path = os.path.join(closed_loop_basepath, method, f"{parameterization}_parameterization", f"agent_update_{agent_update}", f"episode_data_ic_{n_ic}.pkl")
		if method == "untrained":
			clc_path = os.path.join(closed_loop_basepath, method, f"{parameterization}_parameterization", f"episode_data_ic_{n_ic}.pkl")
		elif "td3" in method:
			clc_path = os.path.join(closed_loop_basepath, method, f"episode_data_ic_{n_ic}.pkl")
		if not os.path.exists(clc_path):
			print(f"Closed-loop data not found for method {method} at IC {n_ic}, skipping...")
			continue
		with open(clc_path, "rb") as f:
			episode_data = pickle.load(f)

		closed_loops[method] = episode_data
	
	plot_closed_loop_trajectories(closed_loops, labels = labels, figpath = os.path.join(fig_basepath, f"closed_loop_trajectories_ic_{n_ic}.png"), )
    

100%|██████████| 25/25 [00:48<00:00,  1.93s/it]
